In [2]:
%%capture
!pip install diffusers
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"
!pip install accelerate

In [48]:
import huggingface_hub, diffusers, torch, os, google, itertools, warnings

warnings.filterwarnings('ignore')
google.colab.drive.mount('/content/drive')
WORKING_DIR = "drive/MyDrive/__GGabry/StableDiffusion-utils"

try: os.chdir(WORKING_DIR)
except: pass

assert torch.cuda.get_device_name() == "Tesla T4"
from utils import  StableDiffusionHelper # type: ignore

from env.tokens import HUGGING_FACE_TOKEN
huggingface_hub.hf_api.HfFolder.save_token(HUGGING_FACE_TOKEN)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
!
!git commit -m "rename file" 
! git push origin HEAD:main

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 8.43 KiB | 2.11 MiB/s, done.
Total 4 (delta 0), reused 0 (delta 0)
To https://github.com/GabrieleGhisleni/StableDiffusion-utils.git
   d3d6ebb..60bd40f  HEAD -> main


# Pipeline init

## Schedulers

In [ ]:
lmsd_scheduler = diffusers.LMSDiscreteScheduler(
  beta_start=0.00085, 
    beta_end=0.012, 
    beta_schedule="scaled_linear"
)

In [32]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 17.6 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/command

## Model

In [33]:
%%capture

model_hub = "CompVis/stable-diffusion-v1-4"
model_hub = "stabilityai/stable-diffusion-2-1"


PIPE = diffusers.StableDiffusionPipeline.from_pretrained(
    pretrained_model_name_or_path = model_hub, 
    scheduler = lmsd_scheduler,
    torch_dtype=torch.float16, 
    revision="fp16", 
    use_auth_token=True,
    safety_checker=None
  ).to("cuda")

STH = StableDiffusionHelper(PIPE)

KeyboardInterrupt: ignored

## Generation

In [ ]:
prompts = list(itertools.chain.from_iterable([    
  ["a bee, Cartoon, Artstation, stylish, digital art"] *3,
  ["a magic bee, anime, artstation, digital art"] * 3
]))


N_STEPS = 100
GUIDANCE_SCALES = {3, 7, 12, 30}

STH = StableDiffusionHelper(PIPE)
imgs = STH.generate_images(prompts, GUIDANCE_SCALES, N_STEPS, w=512, h=512)

In [ ]:
STH.display_images_grid(imgs)

In [ ]:
prompts = [
    # "A rose by any other name would smell as sweet. Very beautiful VIEW, Portait, Digital portait, Futuristic, Vaporwave, Digital art",
    # "JOURNEY TO THE DARK ETERNAL EXTREME NORTH. THE WINTER DRAGON GOD IS COMING. DIGITAL ART, foggy, NEON, dystopian, FUTURISTIC"
    "THE HISTORY OF HOW I WAS BORN, DIGITAL ART, FUTURISTIC, artstation",
    # "the continuum flow of life and information represented in the universe; Digital art, futuristic, stone, utopian "
] 

prompts = prompts * 4
print(prompts)


%%bash
type -p curl >/dev/null || sudo apt install curl -y
curl -fsSL https://cli.github.com/packages/githubcli-archive-keyring.gpg | sudo dd of=/usr/share/keyrings/githubcli-archive-keyring.gpg \
&& sudo chmod go+r /usr/share/keyrings/githubcli-archive-keyring.gpg \
&& echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/githubcli-archive-keyring.gpg] https://cli.github.com/packages stable main" | sudo tee /etc/apt/sources.list.d/github-cli.list > /dev/null \
&& sudo apt update \
&& sudo apt install gh -y